In [ ]:
import ipywidgets as widgets 
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext sql
import pandas as pd
import seaborn as sns
import sqlite3

music_genre = pd.read_csv('music_genre.csv')
# query = %sql SELECT track_name from music_genre;
print (music_genre)